In [46]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dropout
from keras.preprocessing import sequence
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import gc
import scipy


In [47]:
os.getcwd()
os.chdir("C:\\Users\\srita\\Python\\Kaggle\\Mercari Price Challenge")

In [48]:
#Read the data
df = pd.read_csv("train.tsv",delimiter='\t',encoding='utf-8')

In [49]:
# Sample the data to speed up computation
df = df.sample(frac=0.1, random_state=10)

df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
774893,774893,Mario and luigi,1,Kids/Toys/Dress Up & Pretend Play,NaN,18.0,1,Girls costumes. I can separate or i will sell ...
431736,431736,Air Jordan Baron 9,3,Men/Shoes/Athletic,Jordan,91.0,0,Sz 10.5 Good preowned condition Includes origi...
1482299,1482299,Dooney & Bourke Wallet,3,Women/Women's Accessories/Wallets,Dooney & Bourke,20.0,1,D&B wallet with credit card slots and back coi...
764342,764342,Lululemon Floral Waistband Speed Tights,3,"Women/Athletic Apparel/Pants, Tights, Leggings",Lululemon,66.0,0,This is a pair of size 6 speed tights in great...
781985,781985,American Eagle Top,2,Women/Athletic Apparel/Shirts & Tops,American Eagle,19.0,0,AEO off the shoulder top size XL


In [50]:
Y = df['price']


## Pre processing

### finding the item description length

In [51]:
df['desc_len'] = df['item_description'].apply(lambda x: len([x for x in str(x).split()]))

In [52]:
df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,desc_len
774893,774893,Mario and luigi,1,Kids/Toys/Dress Up & Pretend Play,NaN,18.0,1,Girls costumes. I can separate or i will sell ...,27
431736,431736,Air Jordan Baron 9,3,Men/Shoes/Athletic,Jordan,91.0,0,Sz 10.5 Good preowned condition Includes origi...,17
1482299,1482299,Dooney & Bourke Wallet,3,Women/Women's Accessories/Wallets,Dooney & Bourke,20.0,1,D&B wallet with credit card slots and back coi...,11
764342,764342,Lululemon Floral Waistband Speed Tights,3,"Women/Athletic Apparel/Pants, Tights, Leggings",Lululemon,66.0,0,This is a pair of size 6 speed tights in great...,53
781985,781985,American Eagle Top,2,Women/Athletic Apparel/Shirts & Tops,American Eagle,19.0,0,AEO off the shoulder top size XL,7


In [53]:
df.drop('train_id', axis=1,inplace=True)
df.drop('name',axis=1,inplace=True)

In [54]:
df.head()

,item_condition_id,category_name,brand_name,price,shipping,item_description,desc_len
774893,1,Kids/Toys/Dress Up & Pretend Play,NaN,18.0,1,Girls costumes. I can separate or i will sell ...,27
431736,3,Men/Shoes/Athletic,Jordan,91.0,0,Sz 10.5 Good preowned condition Includes origi...,17
1482299,3,Women/Women's Accessories/Wallets,Dooney & Bourke,20.0,1,D&B wallet with credit card slots and back coi...,11
764342,3,"Women/Athletic Apparel/Pants, Tights, Leggings",Lululemon,66.0,0,This is a pair of size 6 speed tights in great...,53
781985,2,Women/Athletic Apparel/Shirts & Tops,American Eagle,19.0,0,AEO off the shoulder top size XL,7


In [55]:
Y = df['price']

In [56]:
X = df[['item_condition_id','category_name','brand_name','shipping','item_description']]

In [57]:
X.dtypes
X['item_condition_id'] = X['item_condition_id'].astype('category')
X['shipping'] = X['shipping'].astype('category')

C:\Users\srita\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\srita\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
X['category_name'] = X['category_name'].astype('category')


C:\Users\srita\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [59]:
X['category1'] = X.category_name.str.extract('([^/]+)/[^/]+/[^/]+')
X['subcat2'] = X.category_name.str.extract('[^/]+/([^/]+)/[^/]+')
X['subcat3'] = X.category_name.str.extract('[^/]+/[^/]+/([^/]+)')
X.head()

C:\Users\srita\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
C:\Users\srita\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\srita\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
C:\Users\srita\Anaconda3\lib\site-packages\ipykern

,item_condition_id,category_name,brand_name,shipping,item_description,category1,subcat2,subcat3
774893,1,Kids/Toys/Dress Up & Pretend Play,NaN,1,Girls costumes. I can separate or i will sell ...,Kids,Toys,Dress Up & Pretend Play
431736,3,Men/Shoes/Athletic,Jordan,0,Sz 10.5 Good preowned condition Includes origi...,Men,Shoes,Athletic
1482299,3,Women/Women's Accessories/Wallets,Dooney & Bourke,1,D&B wallet with credit card slots and back coi...,Women,Women's Accessories,Wallets
764342,3,"Women/Athletic Apparel/Pants, Tights, Leggings",Lululemon,0,This is a pair of size 6 speed tights in great...,Women,Athletic Apparel,"Pants, Tights, Leggings"
781985,2,Women/Athletic Apparel/Shirts & Tops,American Eagle,0,AEO off the shoulder top size XL,Women,Athletic Apparel,Shirts & Tops


### average item description 

In [60]:
def avg_words(sentence):
    words = str(sentence).split()
    return (sum(len(word) for word in words)/len(words))
    

In [61]:
X['avg_desc_len'] = X['item_description'].apply(lambda x: avg_words(x))

In [62]:
X[['item_description','avg_desc_len']].head()

,item_description,avg_desc_len
774893,Girls costumes. I can separate or i will sell ...,3.925926
431736,Sz 10.5 Good preowned condition Includes origi...,5.470588
1482299,D&B wallet with credit card slots and back coi...,4.545455
764342,This is a pair of size 6 speed tights in great...,4.113208
781985,AEO off the shoulder top size XL,3.714286


### find the price description in Item description

In [63]:
X['digits_description'] = X['item_description'].apply(lambda x: [x for x in str(x).split() if x.isdigit()])

In [65]:
X[['item_description','digits_description']].head()

,item_description,digits_description
774893,Girls costumes. I can separate or i will sell ...,[4]
431736,Sz 10.5 Good preowned condition Includes origi...,[24]
1482299,D&B wallet with credit card slots and back coi...,[]
764342,This is a pair of size 6 speed tights in great...,[6]
781985,AEO off the shoulder top size XL,[]
